In [2]:
from pony.orm import db_session
from chythonDB.database import db
from chython import smiles
from chythonDB.database import Substance
from multiprocess import Process, Queue
from tqdm import tqdm
import zipfile

Exception ignored in: <function MiniRacer.__del__ at 0x106b0f430>
Traceback (most recent call last):
  File "/Users/timur/University/ChythonDB/.venv/lib/python3.9/site-packages/py_mini_racer/py_mini_racer.py", line 315, in __del__
    self.ext.mr_free_context(getattr(self, "ctx", None))
AttributeError: 'NoneType' object has no attribute 'mr_free_context'


In [2]:
# #If database is not empty and you want to erase it, uncomment this
# from chythonDB.database import db
# from pony.orm import db_session
# from chythonDB.database.config import Config
# db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
#        port=5432)
# db.generate_mapping(check_tables=False, create_tables=True)
# db.drop_all_tables(with_all_data=True)
# db.commit()
# db.disconnect()
# db.unbind()

In [3]:
# Connection to the database
db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
db.generate_mapping(create_tables=True)

In [ ]:
# initialize parameters for fingerprint and LSH
from chythonDB.database.config import Config
db.execute("Create extension if not exists intarray;")
Config(key="fingerprint",value={"min_radius":1, "max_radius":6, "length":2048,
                 "number_active_bits":2, "number_bit_pairs":4})
Config(key="lsh_num_permute",value=64)
Config(key="lsh_threshold",value=0.7)
db.commit()
db.disconnect()
db.unbind()

In [3]:
def worker(q):
    db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
    db.generate_mapping()
    while True:
        data = q.get()
        if data is None:
            break
        for i in data:
            mol, label = i
            mol = smiles(mol)
            mol.canonicalize()
            subs = [(x,None) for x in mol.split()]
            for _ in range(10):
                try:
                    with db_session():
                        Substance(subs)
                        break
                except Exception:
                    continue
    print("finished")

In [4]:
q = Queue(maxsize=30)
num_workers=5
pr = [Process(target=worker, 
              args=(q,)) for _ in range(num_workers)]
[p.start() for p in pr]

[None, None, None, None, None]

The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_FORKED_AND_YOU_CANNOT_USE_THIS_COREFOUNDATION_FUNCTIONALITY___YOU_MUST_EXEC__() to debug.
The process has forked and you cannot use this CoreFoundation functionality safely. You MUST exec().
Break on __THE_PROCESS_HAS_

In [5]:
with zipfile.ZipFile("../dataset/Chembl28.smi.zip", 'r') as zip_ref:
    zip_ref.extractall("")
with open("Chembl28.smi",) as f:
    tmp = []
    for n, row in tqdm(enumerate(f)):
        if n == 50000:
            break
        columns = row.rstrip('\n').lstrip().split(" ")
        smi = columns[0]
        idx = None
        if len(columns) > 1:
            if columns[1].startswith("|"):
                smi += columns[1]
                if len(columns) > 2:
                    idx = columns[2]
            else:
                idx = columns[1]
        if not idx:
            idx = n
        tmp.append((smi, idx))
        if len(tmp) == 1000:
            q.put(tmp)
            tmp = []
    else:
        q.put(tmp)
    for i in range(num_workers):
        q.put(None)

30999it [00:02, 11161.08it/s]


KeyboardInterrupt: 

In [ ]:
db.bind(provider='postgres', user='postgres', host='localhost', password="example", database='test',
        port=5432)
db.generate_mapping()
db.create_fing_index()
db.create_sim_index()